In [1]:
import sys
import os.path
import pikepdf
import urllib.request
import pandas as pd
from datetime import datetime
from pathlib import Path
#import requests
Path("temp").mkdir(parents=True, exist_ok=True)
Path("data").mkdir(parents=True, exist_ok=True)

In [2]:
if (os.path.exists("data/meta.csv")):
    os.remove("data/meta.csv")

In [3]:
#loop
intiger=1
with open("data/PDFURLs.csv") as urls:
    for pdf_url in urls:

        #Creating Unique filename
        unique = pdf_url.removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
        unique = unique.removesuffix("\n")
        metadict = {"Identifier":unique}
        pfad = "temp/" + unique + ".pdf"
        #Try to download file if not in /temp, catch downloaderrors, gesondert 404
        if (not os.path.exists(pfad)):
            print("downloading:", unique)
            try:
                pdf_filename, headers = urllib.request.urlretrieve(pdf_url, pfad)
            except urllib.error.HTTPError as e:
                print("Error while downloading.")
                if e.code == 404:
                    metadict.update({"Error":'Missing(404)'})
                else:
                    metadict.update({"Error":'Unknown Downloaderror'})
                #save Error to meta.csv
                frame = pd.DataFrame(metadict, [0])
                if (os.path.exists("data/meta.csv")):   
                    #oldframe = pd.read_csv("data/meta.csv")
                    newframe = pd.concat([oldframe, frame])
                    newframe.to_csv("data/meta.csv", index=False)
                    oldframe = newframe
                else:
                    frame.to_csv("data/meta.csv", index=False)
                    oldframe = frame
                metadict.clear
                intiger=intiger+1
                continue
        else:
            pdf_filename = pfad
        #try to open pdf, catch Errors
        try:
            pdf = pikepdf.Pdf.open(pdf_filename)           
        except pikepdf._qpdf.PasswordError:
            print("This pdf requires a password.")
            metadict.update({"Error":'PasswordProtected'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/meta.csv")):   
                #oldframe = pd.read_csv("data/meta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/meta.csv", index=False)
                oldframe = newframe
            else:
                frame.to_csv("data/meta.csv", index=False)
                oldframe = frame
            metadict.clear
            intiger=intiger+1
            continue
        except:
            print("An Error occured while opening.")
            metadict.update({"Error":'OpeningError'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/meta.csv")):   
                #oldframe = pd.read_csv("data/meta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/meta.csv", index=False)
                oldframe = newframe
            else:
                frame.to_csv("data/meta.csv", index=False)
                oldframe = frame
            metadict.clear
            intiger=intiger+1
            continue
        
        #Read Metadata from pdfs docinfo
        docinfo = pdf.docinfo

        try:
            docinfo["/Author"]
        except KeyError:
            metadict.update({"Author":""})
        else:
            metadict.update({"Author":str(docinfo["/Author"]).replace("\n", "")\
                                                             .replace("\r", "")})

        try:
            docinfo["/Keywords"]
        except KeyError:
            metadict.update({"Keywords":""})
        else:
            metadict.update({"Keywords":str(docinfo["/Keywords"]).replace("\n", "")\
                                                                 .replace("\r", "")\
                                                                 .replace(";", ",")\
                                                                 .replace(", ", ",")\
                                                                 .replace(" ", "_")})
            
            
        try:
            docinfo["/CreationDate"]
        except KeyError:
            metadict.update({"Datum der Erstellung":""})
        else:
            creation = str(docinfo["/CreationDate"]).replace("'", "")
            
            if(len(creation) > 12):
                if(creation[0] != 'D'):
                    creation = 'D:' + creation
                if(len(creation) >=17):
                    if(creation[16]== 'Z'):
                        creation = creation.removesuffix("0000")
                elif(len(creation) == 16):
                    creation = creation + 'Z'
                try:
                    date = datetime.strptime(creation, "D:%Y%m%d%H%M%S%z")
                except ValueError:
                    metadict.update({"Datum der Erstellung":""})
                    #print("Date Format: ", creation, "\n Maps to:", "unknown")
                else:
                    metadict.update({"Datum der Erstellung":date.strftime("%Y/%m/%d")}) 
                    #print("Known Date Format: ", creation, "\n Maps to:", date)
            else:
                #print("Date to short, dropping: ", creation)
                metadict.update({"Datum der Erstellung":""})
    
        try:
            docinfo["/Title"]
        except KeyError:
            metadict.update({"Titel":""})
        else:
            metadict.update({"Titel":str(docinfo["/Title"]).replace("\n", "")\
                                                           .replace("\r", "")})

            
        metadict.update({"Seitenzahl":len(pdf.pages)})

        
        if (pdf.pages[0].trimbox[2] - pdf.pages[0].trimbox[0]) < (pdf.pages[0].trimbox[3] - pdf.pages[0].trimbox[1]):
            metadict.update({"Format":"Hochformat"})
        else:
            metadict.update({"Format":"Querformat"})
 #old Code zur menschlichen Lesbarkeit der Dateigröße, panda.cut() bietet eine voraussichlich bessere Umsetzung   
#        size = os.stat(pdf_filename).st_size
#        if (size > 1000000000):
#            size = size / 1000000000
#            sizestr =str(round(size, 1)) + " GB"
#        elif (size > 1000000):                    
#            size = size / 1000000
#            sizestr =str(round(size, 1)) + " MB"
#        elif (size > 1000):
#            size = size / 1000
#            sizestr =str(round(size, 1)) + " kB"   
#        else:
#            sizestr =str(round(size, 1)) + " B"
 #       metadict.update({"Dateigröße":str(sizestr)})
        metadict.update({"Dateigröße":os.stat(pdf_filename).st_size})

    
        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("data/meta.csv")):   
            #oldframe = pd.read_csv("data/meta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("data/meta.csv", index=False)
            oldframe = newframe
            
        else:
            frame.to_csv("data/meta.csv", index=False)
            oldframe = frame
        metadict.clear
        pdf.close()
        print("finshed:", intiger)
        intiger=intiger+1

finshed: 1
finshed: 2
finshed: 3
finshed: 4
finshed: 5
finshed: 6
finshed: 7
finshed: 8
finshed: 9
finshed: 10
finshed: 11
finshed: 12
finshed: 13
finshed: 14
finshed: 15
finshed: 16
finshed: 17
finshed: 18
finshed: 19
finshed: 20
finshed: 21
downloading: 7zMn-et47v7I
Error while downloading.
finshed: 23
downloading: 1FDZEIhl8k4M
Error while downloading.
finshed: 25
finshed: 26
finshed: 27
finshed: 28
finshed: 29
finshed: 30
finshed: 31
finshed: 32
finshed: 33
finshed: 34
finshed: 35
finshed: 36
finshed: 37
finshed: 38
finshed: 39
finshed: 40
finshed: 41
finshed: 42
finshed: 43
finshed: 44
finshed: 45
finshed: 46
finshed: 47
finshed: 48
finshed: 49
finshed: 50
finshed: 51
finshed: 52
finshed: 53
finshed: 54
finshed: 55
finshed: 56
finshed: 57
finshed: 58
finshed: 59
finshed: 60
finshed: 61
finshed: 62
finshed: 63
finshed: 64
finshed: 65
finshed: 66
finshed: 67
finshed: 68
finshed: 69
finshed: 70
finshed: 71
finshed: 72
finshed: 73
finshed: 74
finshed: 75
finshed: 76
finshed: 77
finshed

finshed: 557
finshed: 558
finshed: 559
finshed: 560
finshed: 561
finshed: 562
finshed: 563
finshed: 564
finshed: 565
finshed: 566
finshed: 567
finshed: 568
finshed: 569
finshed: 570
finshed: 571
finshed: 572
finshed: 573
finshed: 574
finshed: 575
finshed: 576
finshed: 577
finshed: 578
finshed: 579
finshed: 580
finshed: 581
finshed: 582
finshed: 583
finshed: 584
finshed: 585
finshed: 586
finshed: 587
finshed: 588
finshed: 589
downloading: 7KLTQ1QceXIs
Error while downloading.
downloading: 4ZEHh-SJIkkk
Error while downloading.
finshed: 592
finshed: 593
finshed: 594
finshed: 595
finshed: 596
finshed: 597
downloading: 4BnPeIY8kSeg
Error while downloading.
downloading: 43qisoXs6nFk
Error while downloading.
finshed: 600
finshed: 601
finshed: 602
finshed: 603
finshed: 604
finshed: 605
finshed: 606
finshed: 607
downloading: 113C1PL2j-wc
Error while downloading.
finshed: 609
finshed: 610
downloading: 105UN2ezO6iow
Error while downloading.
finshed: 612
finshed: 613
finshed: 614
finshed: 615
fins

downloading: 4L7gfmmaX1uA
Error while downloading.
downloading: 4eo4dCEGVI0Y
Error while downloading.
downloading: 4wJxs4rcG-T0
Error while downloading.
downloading: 4euuUlfbnIwY
Error while downloading.
downloading: 4RgN8X83RR4s
Error while downloading.
downloading: 42Yvxix8xYzs
Error while downloading.
downloading: 4_ZfTwEyIOB0
Error while downloading.
downloading: 4B9efI_Txay8
Error while downloading.
downloading: 4Tya_S1j0qlM
Error while downloading.
downloading: 4Wol3kKYUklA
Error while downloading.
downloading: 4zT8urXJpKJM
Error while downloading.
downloading: 4XsiUDr_1BQs
Error while downloading.
downloading: 4zhzbErXem9I
Error while downloading.
downloading: 4DdRLkmWw1H8
Error while downloading.
downloading: 4-Ox7gw_ptxA
Error while downloading.
downloading: 4hJn_nWvfjX0
Error while downloading.
downloading: 4jwD2KHVHT2Q
Error while downloading.
downloading: 4behE0mNaFFM
Error while downloading.
downloading: 42QcB6Ycr7WM
Error while downloading.
downloading: 42OcbjBN4e4I
Error

finshed: 1305
finshed: 1306
finshed: 1307
finshed: 1308
finshed: 1309
finshed: 1310
finshed: 1311
finshed: 1312
finshed: 1313
finshed: 1314
finshed: 1315
finshed: 1316
finshed: 1317
finshed: 1318
finshed: 1319
finshed: 1320
finshed: 1321
finshed: 1322
finshed: 1323
finshed: 1324
finshed: 1325
finshed: 1326
downloading: 11NfBNYu72Pj8
Error while downloading.
finshed: 1328
This pdf requires a password.
This pdf requires a password.
This pdf requires a password.
finshed: 1332
This pdf requires a password.
finshed: 1334
finshed: 1335
finshed: 1336
finshed: 1337
finshed: 1338
downloading: 10JFaw-RGwzko
Error while downloading.
downloading: 72WIcANdWfAY
Error while downloading.
finshed: 1341
finshed: 1342
finshed: 1343
finshed: 1344
downloading: 32an6DizCQrk
Error while downloading.
downloading: 3EysWGAX0VnY
Error while downloading.
finshed: 1347
finshed: 1348
finshed: 1349
finshed: 1350
finshed: 1351
downloading: 3F8G70QLeyyE
Error while downloading.
finshed: 1353
finshed: 1354
finshed: 135

Error while downloading.
finshed: 1754
finshed: 1755
downloading: 5Qymu6gAFsTQ
Error while downloading.
finshed: 1757
finshed: 1758
finshed: 1759
finshed: 1760
finshed: 1761
finshed: 1762
finshed: 1763
finshed: 1764
finshed: 1765
finshed: 1766
finshed: 1767
finshed: 1768
finshed: 1769
finshed: 1770
finshed: 1771
finshed: 1772
finshed: 1773
finshed: 1774
finshed: 1775
finshed: 1776
finshed: 1777
finshed: 1778
downloading: 17hJKILNUASz0
Error while downloading.
finshed: 1780
finshed: 1781
finshed: 1782
downloading: 6ZyKSwImz91w
Error while downloading.
finshed: 1784
finshed: 1785
downloading: 9PWFW0nKR2c4
Error while downloading.
finshed: 1787
finshed: 1788
finshed: 1789
finshed: 1790
finshed: 1791
finshed: 1792
finshed: 1793
finshed: 1794
finshed: 1795
finshed: 1796
finshed: 1797
finshed: 1798
finshed: 1799
finshed: 1800
finshed: 1801
finshed: 1802
finshed: 1803
finshed: 1804
finshed: 1805
finshed: 1806
finshed: 1807
finshed: 1808
finshed: 1809
finshed: 1810
finshed: 1811
finshed: 1812


...

finshed: 3270
finshed: 3271
finshed: 3272
finshed: 3273
downloading: 5xxBmsVbfHbc
Error while downloading.
downloading: 5GZnpH-PPdXA
Error while downloading.
downloading: 5J6DLl1ZYF6U
Error while downloading.
downloading: 5ma9mgKIaJEU
Error while downloading.
downloading: 5t9y2mdh06sE
Error while downloading.
downloading: 5JIFSw-quYZE
Error while downloading.
downloading: 57F11YYNgVME
Error while downloading.
finshed: 3281
finshed: 3282
finshed: 3283
finshed: 3284
finshed: 3285
finshed: 3286
finshed: 3287
finshed: 3288
finshed: 3289
finshed: 3290
finshed: 3291
finshed: 3292
finshed: 3293
finshed: 3294
finshed: 3295
finshed: 3296
finshed: 3297
finshed: 3298
finshed: 3299
downloading: 61vQJJgLRGdg
Error while downloading.
finshed: 3301
downloading: 10HGxPUP8Jb6A
Error while downloading.
finshed: 3303
finshed: 3304
finshed: 3305
finshed: 3306
finshed: 3307
finshed: 3308
finshed: 3309
finshed: 3310
finshed: 3311
finshed: 3312
finshed: 3313
finshed: 3314
finshed: 3315
finshed: 3316
finshed:

Error while downloading.
downloading: 3hpbpuS4UjU0
Error while downloading.
downloading: 3MJHoGFmzH0Q
Error while downloading.
downloading: 53Jx86BgxlYs
Error while downloading.
downloading: 53KWc7r6Q7Yg
Error while downloading.
downloading: 3Ne_2YB_oz3g
Error while downloading.
downloading: 3og8p4KLhCek
Error while downloading.
downloading: 63cPzBlT_icg
Error while downloading.
finshed: 3620
finshed: 3621
downloading: 12-r_p0ymAWLc
Error while downloading.
finshed: 3623
finshed: 3624
finshed: 3625
finshed: 3626
finshed: 3627
downloading: 10QdIlLX2UwdY
Error while downloading.
downloading: 10_2B7Zw1tHAI
Error while downloading.
finshed: 3630
downloading: 10uTrVBXkAEC4
Error while downloading.
finshed: 3632
finshed: 3633
downloading: 9GAcsaSIewlE
Error while downloading.
downloading: 9pneVG8IDZOU
Error while downloading.
finshed: 3636
finshed: 3637
finshed: 3638
finshed: 3639
finshed: 3640
finshed: 3641
finshed: 3642
downloading: 6p7b9eRErMEc
Error while downloading.
finshed: 3644
finsh

Error while downloading.
finshed: 4046
downloading: 10oAF4lWlaJ1U
Error while downloading.
finshed: 4048
finshed: 4049
finshed: 4050
finshed: 4051
finshed: 4052
finshed: 4053
finshed: 4054
finshed: 4055
finshed: 4056
finshed: 4057
finshed: 4058
finshed: 4059
finshed: 4060
finshed: 4061
finshed: 4062
finshed: 4063
finshed: 4064
finshed: 4065
finshed: 4066
finshed: 4067
finshed: 4068
finshed: 4069
finshed: 4070
finshed: 4071
finshed: 4072
finshed: 4073
finshed: 4074
finshed: 4075
finshed: 4076
finshed: 4077
downloading: 10aNPBqXULZtY
Error while downloading.
downloading: 10vCTsEHcij4M
Error while downloading.
finshed: 4080
finshed: 4081
downloading: 5WpdSVTbFtQQ
Error while downloading.
finshed: 4083
finshed: 4084
finshed: 4085
downloading: 92ATzsKO5JZo
Error while downloading.
downloading: 7hu3tJhWY7nQ
Error while downloading.
finshed: 4088
finshed: 4089
downloading: 6fmccAed90B8
Error while downloading.
finshed: 4091
finshed: 4092
downloading: 9T-jfAqc8ZqI
Error while downloading.
down

Error while downloading.
downloading: 5K2U5dN6fVnw
Error while downloading.
finshed: 4344
downloading: 7pWzePFaIExA
Error while downloading.
downloading: 437hHU5SdmLY
Error while downloading.
downloading: 40p7cAJJjS4o
Error while downloading.
finshed: 4348
finshed: 4349
finshed: 4350
finshed: 4351
finshed: 4352
finshed: 4353
finshed: 4354
finshed: 4355
finshed: 4356
finshed: 4357
finshed: 4358
finshed: 4359
finshed: 4360
finshed: 4361
finshed: 4362
finshed: 4363
finshed: 4364
finshed: 4365
finshed: 4366
finshed: 4367
finshed: 4368
finshed: 4369
finshed: 4370
finshed: 4371
finshed: 4372
finshed: 4373
finshed: 4374
finshed: 4375
finshed: 4376
finshed: 4377
finshed: 4378
finshed: 4379
finshed: 4380
finshed: 4381
finshed: 4382
finshed: 4383
finshed: 4384
finshed: 4385
finshed: 4386
finshed: 4387
finshed: 4388
finshed: 4389
finshed: 4390
finshed: 4391
finshed: 4392
finshed: 4393
finshed: 4394
finshed: 4395
finshed: 4396
finshed: 4397
finshed: 4398
finshed: 4399
finshed: 4400
finshed: 4401
f

finshed: 4860
finshed: 4861
finshed: 4862
finshed: 4863
downloading: 6uLKk2LHvwEo
Error while downloading.
finshed: 4865
finshed: 4866
finshed: 4867
finshed: 4868
finshed: 4869
finshed: 4870
finshed: 4871
finshed: 4872
finshed: 4873
finshed: 4874
finshed: 4875
downloading: 3T_Y728Hh8kU
Error while downloading.
finshed: 4877
downloading: 3CLbNxizZgzM
Error while downloading.
finshed: 4879
finshed: 4880
finshed: 4881
finshed: 4882
finshed: 4883
finshed: 4884
finshed: 4885
finshed: 4886
finshed: 4887
finshed: 4888
finshed: 4889
finshed: 4890
finshed: 4891
finshed: 4892
finshed: 4893
finshed: 4894
finshed: 4895
finshed: 4896
finshed: 4897
finshed: 4898
finshed: 4899
finshed: 4900
finshed: 4901
finshed: 4902
finshed: 4903
finshed: 4904
finshed: 4905
finshed: 4906
finshed: 4907
finshed: 4908
finshed: 4909
finshed: 4910
finshed: 4911
finshed: 4912
finshed: 4913
finshed: 4914
finshed: 4915
finshed: 4916
downloading: 7UKbEnY0hnTw
Error while downloading.
finshed: 4918
finshed: 4919
finshed: 492